In [1]:
# Import necessary libraries
import os
import pandas as pd
import whylogs
from whylogs.core import DatasetProfileView

# Import your drift detector functions (adjust the module path as needed)
from ..utils.data_drift_detector import (
    save_reference_profile,
    load_reference_profile,
    check_drift_against_reference
)

# Define the path to your products CSV and the reference profile file.
CSV_PATH = r"D:\Anand\Jsearch_ai\data\products.csv"
REF_PROFILE_PATH = "product_reference_profile.bin"


ImportError: attempted relative import with no known parent package

In [2]:
import sys
sys.path.append(r"D:\Anand\Jsearch_ai")


In [3]:
# Import necessary libraries
import os
import pandas as pd
import whylogs
from whylogs.core import DatasetProfileView

# Import your drift detector functions (adjust the module path as needed)
from utils.data_drift_detector import (
    save_reference_profile,
    load_reference_profile,
    check_drift_against_reference
)

# Define the path to your products CSV and the reference profile file.
CSV_PATH = r"D:\Anand\Jsearch_ai\data\products.csv"
REF_PROFILE_PATH = "product_reference_profile.bin"


In [6]:
# Load your product CSV into a DataFrame.
df_products = pd.read_csv(CSV_PATH)
print(f"Loaded data with {len(df_products)} rows and {len(df_products.columns)} columns.")

# Display the first few rows and the column names to know what we have.
df_products.head(), df_products.columns.tolist()


Loaded data with 102637 rows and 10 columns.


(   product_id                                 product_name  \
 0           1                        laboratory equipments   
 1           2                main distribution boards mdbs   
 2           3  motor control centre fixed and drawout type   
 3           4       generator control synchronizing panels   
 4           5                       street lighting panels   
 
                                          description  \
 0  supply installation and commissioning of labor...   
 1  applications protection of whole low voltage e...   
 2  applications controlled starting amp speed con...   
 3  applications controlling generator operation t...   
 4  applications automatic switching of street lig...   
 
                                       category  \
 0  laboratory measuring equipment and supplies   
 1         electrical equipment and accessories   
 2         electrical equipment and accessories   
 3                                          NaN   
 4                   

In [8]:
df_products.isnull().sum()

product_id            0
product_name          2
description          77
category          10475
subcategory       10475
subsubcategory    10475
supplier              0
country               0
state                26
city                  0
dtype: int64

In [9]:
df_products.fillna("Unknown", inplace=True)


In [10]:
df_products.isnull().sum()

product_id        0
product_name      0
description       0
category          0
subcategory       0
subsubcategory    0
supplier          0
country           0
state             0
city              0
dtype: int64

In [11]:
# Check if a reference profile file already exists.
if not os.path.exists(REF_PROFILE_PATH):
    # Create a whylogs profile from your reference data.
    profile_ref = whylogs.log(pandas=df_products).profile().view()
    # Save the reference profile.
    save_reference_profile(profile_ref, REF_PROFILE_PATH)
    print("Reference profile created and saved.")
else:
    # Load the existing reference profile.
    profile_ref = load_reference_profile(REF_PROFILE_PATH)
    print("Reference profile loaded.")


Reference profile created and saved.


In [13]:
# Step 2: Make a copy of the data without any changes.
df_copy = df_products.copy()
print("Data copy created.")



Data copy created.


product_id        0
product_name      0
description       0
category          0
subcategory       0
subsubcategory    0
supplier          0
country           0
state             0
city              0
dtype: int64

In [14]:
# Step 3: Create a reference profile using the unchanged data copy.
profile_ref = whylogs.log(pandas=df_copy).profile().view()
save_reference_profile(profile_ref, REF_PROFILE_PATH)
print("Reference profile created and saved (from unchanged data).")

# Step 4: Compute a new profile from the same unchanged data copy.
profile_new = whylogs.log(pandas=df_copy).profile().view()
print("New profile created (data is unchanged).")

# Step 5: Run the drift detector between the reference and new profile.
# Since the data is identical, we expect no drift.
drift_detected, drift_scores, drifted_columns = check_drift_against_reference(
    new_profile_view=profile_new,
    reference_profile_view=profile_ref,
    with_thresholds=True  # Use raw differences; set True if thresholds are preferred.
)

print("\nDrift Detected:", drift_detected)
print("Drifted Columns:", drifted_columns)
print("Drift Scores:")
for col, metrics in drift_scores.items():
    print(f"  {col}: {metrics}")

Reference profile created and saved (from unchanged data).
New profile created (data is unchanged).

Drift Detected: True
Drifted Columns: ['subcategory', 'state']
Drift Scores:
  product_id: {'algorithm': 'ks', 'pvalue': 0.9999981439048787, 'statistic': 0.0012471136139988737, 'thresholds': {'NO_DRIFT': (0.15, 1), 'POSSIBLE_DRIFT': (0.05, 0.15), 'DRIFT': (0, 0.05)}, 'drift_category': 'NO_DRIFT'}
  product_name: {'algorithm': 'chi-square', 'pvalue': 1.0, 'statistic': 0.0, 'thresholds': {'NO_DRIFT': (0.15, 1), 'POSSIBLE_DRIFT': (0.05, 0.15), 'DRIFT': (0, 0.05)}, 'drift_category': 'NO_DRIFT'}
  description: {'algorithm': 'chi-square', 'pvalue': 1.0, 'statistic': 0.0, 'thresholds': {'NO_DRIFT': (0.15, 1), 'POSSIBLE_DRIFT': (0.05, 0.15), 'DRIFT': (0, 0.05)}, 'drift_category': 'NO_DRIFT'}
  category: {'algorithm': 'chi-square', 'pvalue': 1.0, 'statistic': 0.0006222775357809583, 'thresholds': {'NO_DRIFT': (0.15, 1), 'POSSIBLE_DRIFT': (0.05, 0.15), 'DRIFT': (0, 0.05)}, 'drift_category': 'NO_DR

In [17]:
# Create a DataFrame representing new incoming data
new_data = {
    'product_id': [100001, 100002],
    'product_name': ['cutting edge laser equipment', 'quantum sensor module'],
    'description': ['innovative technology for precision cutting', 'advanced sensor technology for quantum applications'],
    'category': ['advanced laboratory equipment and supplies', 'research and development equipment'],
    'subcategory': ['precision laser systems', 'quantum sensor arrays'],
    'subsubcategory': ['laser cutting modules', 'quantum sensor modules'],
    'supplier': ['new tech innovators llc', 'quantum dynamics inc'],
    'country': ['oman', 'oman'],
    'state': ['muscat', 'muscat'],
    'city': ['muscat', 'muscat']
}
df_new_data = pd.DataFrame(new_data)
print("New incoming data:")
df_new_data
# Append new incoming data (simulate continuous arrival) to the existing product dataset
df_combined = pd.concat([df_products, df_new_data], ignore_index=True)
print(f"Combined data shape: {df_combined.shape}")

# Compute a whylogs profile for the combined (new) data
profile_new = whylogs.log(pandas=df_combined).profile().view()


New incoming data:
Combined data shape: (102639, 10)


In [19]:
# Run the drift detector; adjust 'with_thresholds' as needed for your testing.
drift_detected, drift_scores, drifted_columns = check_drift_against_reference(
    new_profile_view=profile_new,
    reference_profile_view=profile_ref,
    with_thresholds=True  # Toggle to True if using thresholds
)

print("\nDrift Detected:", drift_detected)
print("Drifted Columns:", drifted_columns)
print("Drift Scores:")
for col, metrics in drift_scores.items():
    print(f"  {col}: {metrics}")



Drift Detected: True
Drifted Columns: ['category', 'subcategory']
Drift Scores:
  product_id: {'algorithm': 'ks', 'pvalue': 0.9999973899930293, 'statistic': 0.0012616820453279498, 'thresholds': {'NO_DRIFT': (0.15, 1), 'POSSIBLE_DRIFT': (0.05, 0.15), 'DRIFT': (0, 0.05)}, 'drift_category': 'NO_DRIFT'}
  product_name: {'algorithm': 'chi-square', 'pvalue': 1.0, 'statistic': 0.0, 'thresholds': {'NO_DRIFT': (0.15, 1), 'POSSIBLE_DRIFT': (0.05, 0.15), 'DRIFT': (0, 0.05)}, 'drift_category': 'NO_DRIFT'}
  description: {'algorithm': 'chi-square', 'pvalue': 1.0, 'statistic': 0.0, 'thresholds': {'NO_DRIFT': (0.15, 1), 'POSSIBLE_DRIFT': (0.05, 0.15), 'DRIFT': (0, 0.05)}, 'drift_category': 'NO_DRIFT'}
  category: {'algorithm': 'chi-square', 'pvalue': 0.0, 'statistic': inf, 'thresholds': {'NO_DRIFT': (0.15, 1), 'POSSIBLE_DRIFT': (0.05, 0.15), 'DRIFT': (0, 0.05)}, 'drift_category': 'DRIFT'}
  subcategory: {'algorithm': 'chi-square', 'pvalue': 0.0, 'statistic': inf, 'thresholds': {'NO_DRIFT': (0.15, 1)